# A1. US Presidential Speeches 

In [ ]:
# conda install nltk

In [1]:
# import needed libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import contractions
import string

### Create corpus

In [2]:
from nltk.corpus import inaugural

In [3]:
speech = inaugural.raw('1789-Washington.txt')

### Add year of speech

### Add party affiliation (Democrat/Republican/Other)

### Preprocessing

In [4]:
# expand contractions
expanded_words = [contractions.fix(word) for word in speech.split()]
speech = ' '.join(expanded_words)

In [5]:
# lowercase
speech = speech.lower()

In [6]:
# remove punctuations: using translate
speech = speech.translate(str.maketrans('', '', string.punctuation))

In [7]:
# remove digits and words combined with digits 
speech 

'fellowcitizens of the senate and of the house of representatives among the vicissitudes incident to life no event could have filled me with greater anxieties than that of which the notification was transmitted by your order and received on the 14th day of the present month on the one hand i was summoned by my country whose voice i can never hear but with veneration and love from a retreat which i had chosen with the fondest predilection and in my flattering hopes with an immutable decision as the asylum of my declining years  a retreat which was rendered every day more necessary as well as more dear to me by the addition of habit to inclination and of frequent interruptions in my health to the gradual waste committed on it by time on the other hand the magnitude and difficulty of the trust to which the voice of my country called me being sufficient to awaken in the wisest and most experienced of her citizens a distrustful scrutiny into his qualifications could not but overwhelm with d

In [22]:
# lemmatization
lemmatizer = WordNetLemmatizer()
speech = lemmatizer.lemmatize(speech, pos = 'v')

In [23]:
speech

'fellowcitizens of the senate and of the house of representatives among the vicissitudes incident to life no event could have filled me with greater anxieties than that of which the notification was transmitted by your order and received on the 14th day of the present month on the one hand i was summoned by my country whose voice i can never hear but with veneration and love from a retreat which i had chosen with the fondest predilection and in my flattering hopes with an immutable decision as the asylum of my declining years  a retreat which was rendered every day more necessary as well as more dear to me by the addition of habit to inclination and of frequent interruptions in my health to the gradual waste committed on it by time on the other hand the magnitude and difficulty of the trust to which the voice of my country called me being sufficient to awaken in the wisest and most experienced of her citizens a distrustful scrutiny into his qualifications could not but overwhelm with d

In [24]:
lemmatizer.lemmatize("declining", pos = 'v')

'decline'

In [25]:
wordy = set(nltk.corpus.words.words())

sent = "Io andiamo to the beach with my amico."

" ".join(w for w in nltk.wordpunct_tokenize(sent) \

         if w.lower() in wordy or not w.isalpha())

'Io to the beach with my .'

In [26]:
# remove stopwords
stop_words = set(stopwords.words('english'))

tokens = word_tokenize(speech)

tokens_filtered = [word for word in tokens if not word in stop_words]



In [27]:
# remove extra spaces
speech

'fellowcitizens of the senate and of the house of representatives among the vicissitudes incident to life no event could have filled me with greater anxieties than that of which the notification was transmitted by your order and received on the 14th day of the present month on the one hand i was summoned by my country whose voice i can never hear but with veneration and love from a retreat which i had chosen with the fondest predilection and in my flattering hopes with an immutable decision as the asylum of my declining years  a retreat which was rendered every day more necessary as well as more dear to me by the addition of habit to inclination and of frequent interruptions in my health to the gradual waste committed on it by time on the other hand the magnitude and difficulty of the trust to which the voice of my country called me being sufficient to awaken in the wisest and most experienced of her citizens a distrustful scrutiny into his qualifications could not but overwhelm with d

##### Which presidents have the most/least vocabulary

##### Which president has the least vocabulary

##### On average, do Democratic, Republican, or Other presidents have higher vocabulary?

##### Create a barplot of presidential vocabulary from the earliest president (Washington) to the latest (Trump) in chronological order. Color code this barplot as blue for Democrat, red for Republican, and gray for Others. (1 point)

##### What are the five most frequently used words (exclusive of stop words) used by each president? What are the five most frequently words used collectively by all Democratic presidents versus Republican presidents? (2 point)

##### What are the key themes (e.g., freedom, liberty, country, etc.) used by each president in their inaugural speech? (3 points)

##### Compute a sentiment (positive/negative) for each presidential speech, and draw a barplot of the sentiment of all presidential speeches in chronological order. Again, color code the speeches as blue for Democrat, red for Republican, and gray for Other. Which of these groups have higher mean sentiment score? Who are the top three presidents with the highest positive sentiment in each group? (2 points)